> To live! like a tree alone and free,
> and like a forest in solidarity...
> Nazim Hikmet

# Random Forests



__Objectives__

- Introduction of 'bagging' procedure.

- Identifying the need for bootstrapping for random forests

- Comparing Random forests and bagging methods

- Evaluating a model by random forest model

## Bootstrapping


<img src= "img/bootstrap1.png" style="height:250px">


# Bagging (Boostrapping + Aggregating)


Let's us one more time recall that if $Z_{1}, \cdots, Z_{n}$ are independent observations with variance $\sigma^{2}$ then the variance of the mean $\bar{Z}$ is given by $\frac{\sigma^{2}}{n}$. 

__How is this relevant now?__



We will use this idea calculate $$ \hat{f}^{1}(x), \cdots, \hat{f}^{B}(x)$$ where each $\hat{f}^{i}$ represents a decision tree fitted to the bootstrapped data.

Then we will make a prediction by: 

$$ \hat{f}_{\text{avg}}(x) = \frac{1}{B}\sum_{b=1}^{B} \hat{f}^{b}(x)$$

Note that this is for regression and for the classification we can get majority vote.

_side note: [sklearn averages over probabilities not majority vote](https://scikit-learn.org/stable/modules/ensemble.html#forest)_


## Sklearn for Random Forests

In [11]:
import pandas as pd
import numpy as np

In [2]:
# you can download the data from -- https://www.kaggle.com/ishaanv/ISLR-Auto#Heart.csv

# or http://faculty.marshall.usc.edu/gareth-james/ISL/data.html
heart = pd.read_csv('data/Heart.csv', index_col=0)
heart.head()
print(heart.shape)

(303, 14)


In [3]:
heart.dropna(axis=0, how='any', inplace=True)
y = heart.AHD
X = heart.drop(columns='AHD')

In [4]:
heart.head()

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
1,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
2,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
3,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes
4,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal,No
5,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal,No


In [5]:
heart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 297 entries, 1 to 302
Data columns (total 14 columns):
Age          297 non-null int64
Sex          297 non-null int64
ChestPain    297 non-null object
RestBP       297 non-null int64
Chol         297 non-null int64
Fbs          297 non-null int64
RestECG      297 non-null int64
MaxHR        297 non-null int64
ExAng        297 non-null int64
Oldpeak      297 non-null float64
Slope        297 non-null int64
Ca           297 non-null float64
Thal         297 non-null object
AHD          297 non-null object
dtypes: float64(2), int64(9), object(3)
memory usage: 34.8+ KB


In [6]:
display(heart.Thal.value_counts())
display(heart.ChestPain.value_counts())

normal        164
reversable    115
fixed          18
Name: Thal, dtype: int64

asymptomatic    142
nonanginal       83
nontypical       49
typical          23
Name: ChestPain, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y)

In [8]:
from sklearn.preprocessing import OneHotEncoder

In [9]:
categorical_variables = X_train.select_dtypes(include=['object']).columns
numerical_variables = X_train.select_dtypes(include = ['int64', 'float64']).columns

In [12]:
ohe = OneHotEncoder(drop='first')
X_categ = ohe.fit_transform(X_train[categorical_variables]).toarray()
X_num = X_train[numerical_variables].values
Xtrain = np.concatenate((X_categ, X_num), axis=-1,)
Xtrain.shape

(237, 16)

In [13]:
# now we should transform the test data
# to be able to use it for the prediction

X_test_categ = ohe.transform(X_test[categorical_variables]).toarray()
X_test_num = X_test[numerical_variables].values
Xtest = np.concatenate((X_test_categ, X_test_num), axis=-1,)
Xtest.shape

(60, 16)

In [14]:
from sklearn.ensemble import RandomForestClassifier

In [15]:
clf = RandomForestClassifier(n_estimators=100,
                             criterion='gini',
                             max_features='auto',
                             oob_score=True)

__Your Turn__

- Use 5 fold cross_validation to fit random forest classifier we created above.
- Don't forget to return training scores and trained estimators.

In [16]:
from sklearn.model_selection import cross_validate

In [17]:
cv = 

__Your Turn__

- What is the type of validator above?

- Check test vs train(validation) scores.

- Print "mean +/- std" for both train and test scores

- Also print oob_scores and compare them with cross_validation scores

In [20]:
clf.fit(Xtrain, y_train)
clf.oob_score_

0.8016877637130801

__Your Turn__

- Note that we have over-fitting problem. 

- Let's try to reduce over-fitting

In [ ]:
y_train.value_counts()

In [21]:
clf = RandomForestClassifier(
                             oob_score=True)

clf.fit(Xtrain, y_train)
clf.score(Xtrain, y_train)
clf.oob_score_

0.8059071729957806

### Do it with Pipelines!

In [ ]:
# There is an "easier" way to do this
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier

In [ ]:
numeric_transformer = Pipeline()

categorical_transformer = Pipeline()

preprocessor = ColumnTransformer(
        transformers=[
        ('num', numeric_transformer, numerical_variables),
        ('cat', categorical_transformer, categorical_variables)],)

rf = Pipeline(??)

In [ ]:
pipe_validator = cross_validate(???)

In [ ]:
# train scores
print(cv['train_score'])
# validation scores
print(cv['test_score'])

# let's pick one of the estimator for further investigation

est = cv['estimator'][0]

In [ ]:
est['classifier'].oob_score_

## Feature Importance

In [22]:
feature_importances = clf.feature_importances_

In [23]:
feature_importances

array([0.03342457, 0.01191084, 0.01911239, 0.08933508, 0.07189635,
       0.08474122, 0.02560453, 0.0772918 , 0.07266779, 0.01008271,
       0.02141654, 0.13129551, 0.06641991, 0.11646117, 0.04589663,
       0.122443  ])

In [24]:
# be careful with the order of columns
columns = ohe.get_feature_names().tolist() + numerical_variables.tolist()

In [25]:
importances = pd.DataFrame(data=feature_importances,
                           index=columns, columns=['feature_importances'])

importances.sort_values(by='feature_importances', ascending=False)

,feature_importances
MaxHR,0.131296
Ca,0.122443
Oldpeak,0.116461
x1_normal,0.089335
Age,0.084741
RestBP,0.077292
Chol,0.072668
x1_reversable,0.071896
ExAng,0.066420
Slope,0.045897


### Extra Material 

- [Sklearn averages probabilities in RF implementation](https://scikit-learn.org/stable/modules/ensemble.html#forest)

- [On the variance](https://newonlinecourses.science.psu.edu/stat414/node/167/)

- [Is RF immune to overfitting?](https://en.wikipedia.org/wiki/Talk%3ARandom_forest)

- [Tricky stuff with respect to feature importance](http://rnowling.github.io/machine/learning/2015/08/10/random-forest-bias.html)

- [An interesting implementation of feature importance](https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances_faces.html#sphx-glr-auto-examples-ensemble-plot-forest-importances-faces-py)

- [Different Ensemble Methods in sklearn](https://scikit-learn.org/stable/modules/ensemble.html#forest)

- [ISLR - section 8.2](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf)

- [Another library for RF: H2o](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/drf.html)